In [22]:
from bioio import BioImage
from bioio.writers import OmeTiffWriter
import bioio_nd2
import os
from glob import glob
import numpy as np
import pandas as pd
import skimage as sk
import napari

In [15]:
def replace_spaces_with_underscores_recursive(directory):
    """
    Recursively replaces spaces with underscores in file and folder names
    within the specified directory.
    """
    for root, dirs, files in os.walk(directory):
        # Rename files
        for filename in files:
            if ' ' in filename:
                old_path = os.path.join(root, filename)
                new_filename = filename.replace(' ', '_')
                new_path = os.path.join(root, new_filename)
                try:
                    os.rename(old_path, new_path)
                    print(f"Renamed file: {old_path} -> {new_path}")
                except OSError as e:
                    print(f"Error renaming file {old_path}: {e}")

        # Rename directories (important to do this after files in the current root
        # to avoid issues with paths changing during file renaming)
        for i in range(len(dirs)):
            dirname = dirs[i]
            if ' ' in dirname:
                old_path = os.path.join(root, dirname)
                new_dirname = dirname.replace(' ', '_')
                new_path = os.path.join(root, new_dirname)
                try:
                    os.rename(old_path, new_path)
                    print(f"Renamed directory: {old_path} -> {new_path}")
                    # Update the list of directories so os.walk continues correctly
                    dirs[i] = new_dirname
                except OSError as e:
                    print(f"Error renaming directory {old_path}: {e}")

In [16]:
directory = "E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/"
replace_spaces_with_underscores_recursive(directory)

Renamed directory: E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day 1 adult animals -> E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day_1_adult_animals
Renamed directory: E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day 7 adult animals -> E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day_7_adult_animals
Renamed directory: E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/L4 animals -> E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/L4_animals
Renamed file: E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day_1_adult_animals\D1A_rep 1.nd2 -> E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day_1_adult_animals\D1A_rep_1.nd2
Renamed file: E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day_1_adult_animals\D1A_rep 10.nd2 -> E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day_1_adult_animals\D1A_rep_10.nd2
Renamed file: E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/Day_1_adult_animals\D1A_rep 11.nd2 -> E:/

In [ ]:
file_list = sorted(glob(r"E:/Patel_Lab/OIC-201_Analyzing_mito_content_mtDNA/Data/**/*.nd2"))
images = [BioImage(file, reader=bioio_nd2.Reader) for file in file_list]

In [32]:
images[0].dims.order

'TCZYX'

In [19]:
img_data = [img.data for img in images]


In [31]:
img_squeeze = np.squeeze(img_data[0])
img_squeeze.shape

(2, 7, 704, 704)

In [21]:
viewer = napari.view_image(img_data[0])

In [ ]:
z = images[0].physical_pixel_sizes.Z
x = images[0].physical_pixel_sizes.X
y = images[0].physical_pixel_sizes.Y
save_path = "./Data/Tiffs/"
for i in range(len(img_data)):
    name = os.path.basename(file_list[i])
    squeeze = np.squeeze(img_data[i])
    sk.io.imsave(os.path.join(save_path,name[:-4]+'.tif'), squeeze, check_contrast=False)

In [23]:
help(OmeTiffWriter)

Help on class OmeTiffWriter in module bioio_ome_tiff.writers.ome_tiff_writer:

class OmeTiffWriter(bioio_base.writer.Writer)
 |  Method resolution order:
 |      OmeTiffWriter
 |      bioio_base.writer.Writer
 |      abc.ABC
 |      builtins.object
 |  
 |  Static methods defined here:
 |  
 |  build_ome(data_shapes: List[Tuple[int, ...]], data_types: List[numpy.dtype], dimension_order: Optional[List[Optional[str]]] = None, channel_names: Optional[List[Optional[List[str]]]] = None, image_name: Optional[List[Optional[str]]] = None, physical_pixel_sizes: Optional[List[bioio_base.types.PhysicalPixelSizes]] = None, channel_colors: Optional[List[Optional[List[List[int]]]]] = None) -> ome_types._autogenerated.ome_2016_06.ome.OME
 |      Create the necessary metadata for an OME tiff image
 |      
 |      Parameters
 |      ----------
 |      data_shapes:
 |          A list of 5- or 6-d tuples
 |      data_types:
 |          A list of data types
 |      dimension_order:
 |          The order 

In [ ]:
#Read in 